# 4.1 다층 퍼셉트론



In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class MultilayerPerceptron(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):

    super(MultilayerPerceptron, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x_in, apply_softmax=False):

    intermediate = F.relu(self.fc1(x_in))
    output = self.fc2(intermediate)

    if apply_softmax:
      output = F.softmax(output, dim=1)
    return output

In [3]:
batch_size = 2
input_dim = 3
hidden_dim = 100
output_dim = 4

mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


In [4]:
def describe(x):
  print('타입: {}'.format(x.type()))
  print('크기: {}'.format(x.shape))
  print('값: {}'.format(x))

In [5]:
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: tensor([[0.2942, 0.0211, 0.2198],
        [0.5267, 0.1835, 0.5192]])


In [6]:
y_output = mlp(x_input, apply_softmax=False)
# y_output = mlp.foward(x_input, apply_softmax=False)

describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: tensor([[ 0.1104, -0.1960,  0.2280,  0.1792],
        [ 0.1731, -0.1726,  0.1553,  0.1182]], grad_fn=<AddmmBackward0>)


In [7]:
y_output = mlp(x_input, apply_softmax=True)

describe(y_output)
print(torch.sum(y_output, dim=1))

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: tensor([[0.2543, 0.1872, 0.2861, 0.2724],
        [0.2750, 0.1946, 0.2701, 0.2603]], grad_fn=<SoftmaxBackward0>)
tensor([1., 1.], grad_fn=<SumBackward1>)


# 4.2 예제: MLP로 성씨 분류하기

In [8]:
!mkdir data
!wget https://git.io/JtaFp -O data/download.py
!wget https://git.io/Jtabe -O data/get-all-data.sh
!chmod 755 data/get-all-data.sh
%cd data
!./get-all-data.sh
%cd ..

--2024-02-03 15:48:32--  https://git.io/JtaFp
Resolving git.io (git.io)... 140.82.113.21
Connecting to git.io (git.io)|140.82.113.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_4/4_2_mlp_surnames/data/download.py [following]
--2024-02-03 15:48:32--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_4/4_2_mlp_surnames/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘data/download.py’

data/download.py    100%[===================>]   1.54K  --.-KB/s    in 0s      

2024-02-03 15:48:32 (24.0 MB/s) - ‘data/download.py’ saved [1572/1572]

--2024-02-03 15:48:32--  htt

In [9]:
from argparse import Namespace
from collections import Counter
import json
import os
import string

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

In [10]:
class Vocabulary(object):

  def __init__(self, token_to_idx=None, add_unk=True, unk_token='<UNK>'):

    if token_to_idx is None:
        token_to_idx = {}
    self._token_to_idx = token_to_idx

    self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

    self._add_unk = add_unk
    self._unk_token = unk_token

    self.unk_index = -1
    if add_unk:
        self.unk_index = self.add_token(unk_token)


    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환합니다 """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}


  def to_serializable(self):
    return {'token_to_idx': self._token_to_idx,
            'add_unk': self._add_unk,
            'unk_token': self._unk_token}

  @classmethod
  def from_serializable(cls, contents):
    return cls(**contents)

  def add_token(self, token):
    try:
        index = self._token_to_idx[token]
    except KeyError:
        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
    return index

  def add_many(self, tokens):
    return [self.add_token(token) for token in tokens]

  def lookup_token(self, token):
    if self.unk_index >= 0:
        return self._token_to_idx.get(token, self.unk_index)
    else:
        return self._token_to_idx[token]

  def lookup_index(self, index):
    if index not in self._idx_to_token:
      raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
    return self._idx_to_token[index]

  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)

  def __len__(self):
    return len(self._token_to_idx)

In [11]:
class SurnameVectorizer(object):
  def __init__(self, surname_vocab, nationality_vocab):
    self.surname_vocab = surname_vocab
    self.nationality_vocab = nationality_vocab

  def vectorize(self, surname):
    vocab = self.surname_vocab
    one_hot = np.zeros(len(vocab), dtype=np.float32)
    for token in surname:
        one_hot[vocab.lookup_token(token)] = 1

    return one_hot

  @classmethod
  def from_dataframe(cls, surname_df):
    surname_vocab = Vocabulary(unk_token='@')
    nationality_vocab = nationality_vocab = Vocabulary(add_unk=False)

    for index, row in surname_df.iterrows():
        for letter in row.surname:
            surname_vocab.add_token(letter)
        nationality_vocab.add_token(row.nationality)

    return cls(surname_vocab, nationality_vocab)

  @classmethod
  def from_serializable(cls, contents):
      surname_vocab = Vocabulary.from_serializable(contents['surname_vocab'])
      nationality_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])
      return cls(surname_vocab=surname_vocab, nationality_vocab=nationality_vocab)

  def to_serializable(self):
      return {'surname_vocab': self.surname_vocab.to_serializable(),
              'nationality_vocab': self.nationality_vocab.to_serializable()}

In [19]:
class SurnameDataset(Dataset):
  def __init__(self, surname_df, vectorizer):
    self.surname_df = surname_df
    self._vectorizer = vectorizer

    self.train_df = self.surname_df[self.surname_df.split=='train']
    self.train_size = len(self.train_df)

    self.val_df = self.surname_df[self.surname_df.split=='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.surname_df[self.surname_df.split=='test']
    self.test_size = len(self.test_df)

    self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

    self.set_split('train')


    class_counts = surname_df.nationality.value_counts().to_dict()
    def sort_key(item):
        return self._vectorizer.nationality_vocab.lookup_token(item[0])
    sorted_counts = sorted(class_counts.items(), key=sort_key)
    frequencies = [count for _, count in sorted_counts]
    self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)


  @classmethod
  def load_dataset_and_make_vectorizer(cls, surname_csv):
    surname_df = pd.read_csv(surname_csv)
    train_surname_df = surname_df[surname_df.split=='train']
    return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))



  @classmethod
  def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath):
    surname_df = pd.read_csv(surname_csv)
    vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
    return cls(surname_df, vectorizer)


  @staticmethod
  def load_vectorizer_only(vectorizer_filepath):

    with open(vectorizer_filepath) as fp:
        return SurnameVectorizer.from_serializable(json.load(fp))


  def save_vectorizer(self, vectorizer_filepath):
    with open(vectorizer_filepath, "w") as fp:
        json.dump(self._vectorizer.to_serializable(), fp)


  def get_vectorizer(self):
    return self._vectorizer


  def set_split(self, split="train"):
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]


  def __len__(self):
    return self._target_size


  def __getitem__(self, index):

    row = self._target_df.iloc[index]

    surname_vector = \
        self._vectorizer.vectorize(row.surname)

    nationality_index = \
        self._vectorizer.nationality_vocab.lookup_token(row.nationality)

    return {'x_surname': surname_vector,
            'y_nationality': nationality_index}


  def get_num_batches(self, batch_size):
      return len(self) // batch_size


def generate_batches(dataset, batch_size, shuffle=True,
                        drop_last=True, device="cpu"):

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
             out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict


In [20]:
import torch.nn as nn
import torch.nn.functional as F

class SurnameClassifier(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(SurnameClassifier,self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x_in, apply_softmax=False):
    intermediate = F.relu(self.fc1(x_in))
    prediction_vector = self.fc2(intermediate)

    if apply_softmax:
      prediction_vector = F.softmax(prediction_vector)

    return prediction_vector

In [21]:
args = Namespace(
    surname_csv = 'data/surnames/surnames_with_splits.csv',
    vectorizer_file = 'vectorizer.json',
    model_state_file = 'model.pth',
    save_dir = 'surname_mlp',
    hidden_dim=300,
    seed = 22,
    num_epochs=100,
    early_stopping_criteria=5,
    learning_rate=1e-3,
    batch_size = 64,
    cuda=False,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True
)

In [22]:
def make_train_state(args):
  return {'stop_early': False,
          'early_stopping_step': 0,
          'early_stopping_best_val': 1e8,
          'learning_rate': args.learning_rate,
          'epoch_index':0,
          'train_loss': [],
          'train_acc': [],
          'val_loss': [],
          'val_acc': [],
          'test_loss': -1,
          'test_acc': -1,
          'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
  if train_state['epoch_index'] == 0:
    torch.save(model.state_dict(), train_state['model_filename'])
    train_state['stop_early'] = False

  elif train_state['epoch_index'] >=1:
    loss_tm1, loss_t = train_state['val_loss'][-2:]

    if loss_t >= train_state['early_stopping_best_val']:
      train_state['early_stopping_step'] +=1
    else:

      if loss_t < train_state['early_stopping_best_val']:
        torch.save(model.state_dict(), train_state['model_filename'])

      train_state['early_stopping_step'] = 0

    train_state['stop_early'] = train_state['early_stopping_step'] >= args.early_stopping_criteria

  return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [23]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [24]:
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

    print("파일 경로: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

print("CUDA 사용여부: {}".format(args.cuda))

set_seed_everywhere(args.seed, args.cuda)

handle_dirs(args.save_dir)

파일 경로: 
	surname_mlp/vectorizer.json
	surname_mlp/model.pth
CUDA 사용여부: False


In [25]:
if args.reload_from_files:
    # 체크포인트에서 훈련을 다시 시작
    print("로딩!")
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
else:
    # 데이터셋과 Vectorizer 만들기
    print("새로 만들기!")
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()
classifier = SurnameClassifier(input_dim=len(vectorizer.surname_vocab),
                               hidden_dim=args.hidden_dim,
                               output_dim=len(vectorizer.nationality_vocab))

새로 만들기!


In [26]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)


loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm.notebook.tqdm(desc='training routine',
                               total=args.num_epochs,
                               position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                               total=dataset.get_num_batches(args.batch_size),
                               position=1,
                               leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                             total=dataset.get_num_batches(args.batch_size),
                             position=1,
                             leave=True)


for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index


    dataset.set_split('train')
    batch_generator = generate_batches(dataset,
                                       batch_size=args.batch_size,
                                       device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()

    for batch_index, batch_dict in enumerate(batch_generator):

        optimizer.zero_grad()

        y_pred = classifier(batch_dict['x_surname'])

        loss = loss_func(y_pred, batch_dict['y_nationality'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        loss.backward()

        optimizer.step()

        acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

        train_bar.set_postfix(loss=running_loss, acc=running_acc,
                            epoch=epoch_index)
        train_bar.update()

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)


    dataset.set_split('val')
    batch_generator = generate_batches(dataset,
                                       batch_size=args.batch_size,
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):

        y_pred =  classifier(batch_dict['x_surname'])

        loss = loss_func(y_pred, batch_dict['y_nationality'])
        loss_t = loss.to("cpu").item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)
        val_bar.set_postfix(loss=running_loss, acc=running_acc,
                            epoch=epoch_index)
        val_bar.update()

    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

    train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

    scheduler.step(train_state['val_loss'][-1])

    if train_state['stop_early']:
        break

    train_bar.n = 0
    val_bar.n = 0
    epoch_bar.update()

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/120 [00:00<?, ?it/s]

split=val:   0%|          | 0/25 [00:00<?, ?it/s]

In [27]:
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    y_pred =  classifier(batch_dict['x_surname'])

    loss = loss_func(y_pred, batch_dict['y_nationality'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [28]:
print("테스트 손실: {};".format(train_state['test_loss']))
print("테스트 정확도: {}".format(train_state['test_acc']))

테스트 손실: 1.7984622812271118;
테스트 정확도: 46.68750000000001


In [29]:
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).view(1, -1)
    result = classifier(vectorized_surname, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality, 'probability': probability_value}

In [30]:
new_surname = input("분류하려는 성씨를 입력하세요: ")
classifier = classifier.to("cpu")
prediction = predict_nationality(new_surname, classifier, vectorizer)
print("{} -> {} (p={:0.2f})".format(new_surname,
                                    prediction['nationality'],
                                    prediction['probability']))

분류하려는 성씨를 입력하세요: ㅡㅡㅡ
ㅡㅡㅡ -> Korean (p=0.36)


<ipython-input-20-0d748e171aa1>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prediction_vector = F.softmax(prediction_vector)


In [31]:
vectorizer.nationality_vocab.lookup_index(8)

'Irish'

In [33]:
def predict_topk_nationality(name, classifier, vectorizer, k=5):

    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).view(1, -1)
    prediction_vector = classifier(vectorized_name, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k)

    # 반환되는 크기는 (1,k)입니다
    probability_values = probability_values.detach().numpy()[0]
    indices = indices.detach().numpy()[0]

    results = []
    for prob_value, index in zip(probability_values, indices):
        nationality = vectorizer.nationality_vocab.lookup_index(index)
        results.append({'nationality': nationality,
                        'probability': prob_value})

    return results


new_surname = input("분류하려는 성씨를 입력하세요: ")
classifier = classifier.to("cpu")

k = int(input("얼마나 많은 예측을 보고 싶나요? "))
if k > len(vectorizer.nationality_vocab):
    print("앗! 전체 국적 개수보다 큰 값을 입력했습니다. 모든 국적에 대한 예측을 반환합니다. :)")
    k = len(vectorizer.nationality_vocab)

predictions = predict_topk_nationality(new_surname, classifier, vectorizer, k=k)

print("최상위 {}개 예측:".format(k))
print("===================")
for prediction in predictions:
    print("{} -> {} (p={:0.2f})".format(new_surname,
                                        prediction['nationality'],
                                        prediction['probability']))

분류하려는 성씨를 입력하세요: suzuki
얼마나 많은 예측을 보고 싶나요? 5
최상위 5개 예측:
suzuki -> Polish (p=0.81)
suzuki -> Japanese (p=0.11)
suzuki -> Russian (p=0.03)
suzuki -> Czech (p=0.02)
suzuki -> Greek (p=0.01)


<ipython-input-20-0d748e171aa1>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prediction_vector = F.softmax(prediction_vector)
